In [1]:
# IMPORT LIBRARIES
import psycopg2 as pg2
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import snowflake.connector

In [2]:
#CONNECT TO SNOWFLAKE WITH REPORTER CREDENTIALS
conn = snowflake.connector.connect(
    user='emirkabasoglu', #emirkabasoglu
    password='7Bb13287', #7Bb13287
    account='se63477.north-europe.azure',
    warehouse='TRANSFORMING_WH',
    database='PROD_ANALYTICS_DB'
    )

In [3]:
#START THE CURSOR AND RUN THE QUERY
cur = conn.cursor()
cur.execute(''' --BOARD METRICS

SELECT 
t2.economic_id as id,
t2.customer_id as customer_id,
t2.customer_name,
t1.item_id,
t5.external_name,
t5.internal_name,
t1.quantity,
t1.arr_eur as value,
t1.committed_date as commit,
t1.start_date as "start",
t1.end_date   as end ,
CASE WHEN t1.status IN ('cancelled','non_renewing') then 'y' else null end as cancelled,
t3.region,
CASE WHEN t4.fte_size < 250 THEN 'smb' 
     WHEN (t4.fte_size > 249 AND 5001 > t4.fte_size)  THEN 'mm'
     WHEN t4.fte_size IS NULL THEN NULL
     ELSE 'ent' END AS size,
CASE WHEN t5.item_family_id IN ('3','4') OR T1.item_id IN ('8_2','8_3') THEN 'content' 
     ELSE 'software' END AS product,
CASE WHEN t5.item_family_id IN ('2') or t1.item_id IN ('8_4','8_5') THEN 'infinite'
     WHEN t5.item_family_id IN ('3') THEN ''
     ELSE 'people' END AS type,
CASE WHEN t6.inbound_lead_source IN ('PPC - Google') then 'google'
     WHEN t6.inbound_lead_source IN ('PPC - Bing') then 'bing' 
     WHEN t6.inbound_lead_source IN ('PPC - Facebook') then 'facebook'
     WHEN t6.inbound_lead_source IN ('PPC - Linkedin') then 'linkedin'       
     WHEN t6.inbound_lead_source IN ('Review DB - Capterra','Review DB - Software Advice','Review DB - Getapp', 'Review DB - eLearning Industry', 'Review DB - People Managing People', 'Review DB - Select Software', 'Review DB - Softwaresuggest') then 'review' 
     WHEN t6.inbound_lead_source IN ('Referral - Other') then 'other'
     ELSE null END AS inbound_source

FROM dbt_deploy.subscriptions t1
LEFT JOIN dbt_deploy.customers t2 on t1.customer_id = t2.customer_id 
LEFT JOIN dbt_deploy.countries_and_regions t3 on t2.billing_address_country =t3.country_id
LEFT JOIN dbt_deploy.stg_hubspot__company t4 on t4.company_id = t2.hubspot_company_id
LEFT JOIN dbt_deploy.products t5 on t1.item_id = t5.item_id
LEFT JOIN dbt_deploy.stg_hubspot__deal t6 on t6.deal_id = t1.order_id
ORDER BY 1    

''')


In [4]:
# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)

In [5]:
# ASSIGN TO MAIN DF FOR FUTHER TRANSFORMATION
df_main = df_items_db_fetched

In [6]:
df_main.columns

Index(['ID', 'CUSTOMER_ID', 'CUSTOMER_NAME', 'ITEM_ID', 'EXTERNAL_NAME',
       'INTERNAL_NAME', 'QUANTITY', 'VALUE', 'COMMIT', 'start', 'END',
       'CANCELLED', 'REGION', 'SIZE', 'PRODUCT', 'TYPE', 'INBOUND_SOURCE'],
      dtype='object')

In [7]:
#CONVERT ALL THE COLUMN NAMES LOWER CASE
df_main.columns= df_main.columns.str.lower()
df_main['size'] = df_main['size'].str.lower()
df_main['region'] = df_main['region'].str.lower()



In [8]:
df_main.head(2)

,id,customer_id,customer_name,item_id,external_name,internal_name,quantity,value,commit,start,end,cancelled,region,size,product,type,inbound_source
0,225,7734TT5L9QCL25O,ISAL hf,1_17,People Service Sms Gateway Access Setup,people_service_sms_gateway_access_setup,1,0.0,2018-11-16,2019-01-01,2022-12-31,y,norben,mm,software,people,None
1,225,7734TT5L9QCL25O,ISAL hf,1_1,People Learning Management,people_lms,1000,14762.0,2018-11-16,2019-01-01,2022-12-31,y,norben,mm,software,people,None


In [9]:
# OUTPUT FINAL FILE WITH ALL DIFFERENCES
df_main.to_excel('Board_Metrics_Input_File.xlsx')